In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#database values
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

In [3]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

usedu_df = pd.read_sql_query('select * from useducation',con=engine)
# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [4]:
#remove poorly reporting jurisdictions
def remove_nonstate(df):
    df = df[df.ENROLL.notna()]
    return df

usedu_stateonly_df = remove_nonstate(usedu_df)


In [5]:
#state groups to see where missing value are and plot
def plot_nulls(datasource):
    datasource = datasource.groupby('STATE').apply(lambda x: x.isna().sum() / x.count())#.transpose()
    sns.set(style="whitegrid", palette="pastel")
    f2, ax2 = plt.subplots(figsize=(15, 7))
    ax2 = sns.heatmap(datasource, vmin=0, vmax=1)
#plot_nulls(usedu_df) 

In [6]:
#plot_nulls(usedu_stateonly_df)

In [7]:
usedu_stateonly_df = usedu_stateonly_df.sort_values(by=["YEAR"]).groupby('STATE').apply(lambda x: x.interpolate())

In [8]:
#plot_nulls(usedu_stateonly_df)

In [9]:
#usedu_stateonly_df[usedu_stateonly_df.isna().any(axis=1)]

In [10]:
#replace remaining nulls with state mean for column
def statecolmean(df):
    for column in df.columns:
        try:
            df[column] = df.groupby('STATE')[column].transform(lambda x: x.fillna(x.mean()))
        except TypeError:
            pass
statecolmean(usedu_stateonly_df)    

In [11]:
#plot_nulls(usedu_stateonly_df)